# **💁🏻🗨️💁🏻‍♂️대화 요약 SOLAR API code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 자료에서는 Solar Chat API를 이용하여 대화 요약 대회를 풀어봅니다.     

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [ ]:
# !pip install openai

In [2]:
import pandas as pd
import os
import time
from tqdm import tqdm
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.
from openai import OpenAI # openai==1.2.0
from dotenv import load_dotenv


### 2) Solar Chat API Client 생성하기
- 앞으로 Solar Chat API를 사용하기 위해 Client를 생성합니다.

In [3]:
ENV_PATH = '/data/ephemeral/home/NLP_JES/llm/.env'
load_dotenv(ENV_PATH)
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY") # upstage.ai에서 발급받은 API KEY를 입력해주세요.

client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1/solar"
)

### 3) Solar Chat API 사용해보기 (선택)
- 예시 코드를 통해 Solar Chat API를 사용해보세요.

In [69]:
stream = client.chat.completions.create(
    model="solar-1-mini-chat",
    messages=[
      {
        "role": "system",
        "content": "You are a helpful assistant."
      },
      {
        "role": "user",
        "content": "Hello!"
      }
    ],
    stream=True,
)
 
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
 
# Use with stream=False
# print(stream.choices[0].message.content)

Hello! How can I assist you today?

### 4) 데이터 불러오기
- 실험에서 쓰일 데이터를 load합니다.

In [4]:
# 데이터 경로를 지정해줍니다.
BASE_PATH = '/data/ephemeral/home/NLP_JES'
LLM_PATH = os.path.join(BASE_PATH, 'llm')
ENV_PATH = os.path.join(LLM_PATH, '.env')
DATA_PATH = '/data/ephemeral/home/baseline/data'
RESULT_PATH = os.path.join(BASE_PATH, 'llm', 'result')

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(DATA_PATH,'train.csv'))
train_df.tail()

,fname,dialogue,summary,topic
12452,train_12455,#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직
12456,train_12459,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기


In [5]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(DATA_PATH,'dev.csv'))
val_df.tail()

,fname,dialogue,summary,topic
494,dev_495,#Person1#: 이제 새해가 되어서 새로운 시작을 하려고 결심했어. \r\n#P...,#Person1#은 새해에 금연을 하고 커밍아웃하기로 결정했습니다. #Person2...,새해
495,dev_496,"#Person1#: 너, 조랑 결혼했지? \r\n#Person2#: 조? 무슨 말인...",#Person1#은 #Person2#가 조와 결혼했다고 생각했다. #Person2#...,사랑에 빠지다
496,dev_497,"#Person1#: 무엇을 도와드릴까요, 부인?\r\n#Person2#: 몇 주 동...",#Person2#의 차에서 이상한 소리가 납니다. #Person1#는 브레이크를 교...,소음
497,dev_498,"#Person1#: 안녕하세요, 아마존 고객 서비스입니다. 무엇을 도와드릴까요?\n...",#Person2#님이 아마존 고객 서비스에 전화하여 아마존에서 받은 책에 한 페이지...,빠진 페이지
498,dev_499,#Person1#: 여름이 다 되어간다는 게 믿기지 않아.\r\n#Person2#:...,#Person2#는 #Person1#에게 여름 휴가 동안 파티를 도와주는 회사에서 ...,여름 휴가


In [72]:
test_df = pd.read_csv(os.path.join(DATA_PATH,'test.csv'))
# display(test_df.tail())
result_df = pd.read_csv(os.path.join(RESULT_PATH, 'solar_test_output.csv'))
display(result_df.head())

,fname,summary
0,test_0,"#Person1#은 모든 사무실 통신을 이메일과 공식 메모로 제한하고, 근무 시간 ..."
1,test_1,#Person2#는 교통 체증으로 인해 집에 늦게 도착했습니다. #Person1#은...
2,test_2,#Person1#은 마샤와 히어로가 이혼하려고 한다고 #Person2#에게 알렸습니...
3,test_3,브라이언의 생일 파티가 열리고 있습니다. #Person1#은 브라이언에게 목걸이를 ...
4,test_4,#Person1#은 #Person2#에게 올림픽 공원의 크기에 대해 언급합니다. #...


## 1. Solar Chat API 요약 성능 확인하기
- Solar Chat API을 이용하여 train 및 validation dataset에 포함된 dialogue 샘플을 요약해 봅니다.

In [73]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
rouge = Rouge()
def compute_metrics(pred, gold):
    results = rouge.get_scores(pred, gold, avg=True)
    result = {key: value["f"] for key, value in results.items()}
    return result

In [21]:
# Dialogue를 입력으로 받아, Solar Chat API에 보낼 Prompt를 생성하는 함수를 정의합니다.
def build_prompt(dialogue):
    system_prompt = "You are an expert in the field of dialogue summarization. Please summarize the following dialogue."

    user_prompt = f"Dialogue:\n{dialogue}\n\nSummary:\n"
    
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]

In [22]:
# Solar Chat API를 활용해 Summarization을 수행하는 함수를 정의합니다.
def summarization(dialogue):
    summary = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=build_prompt(dialogue),
    )

    return summary.choices[0].message.content

### (선택) parameter 변경하기
- Solar Chat API를 사용할 때, parameter를 변경하여, 다양한 결과를 얻을 수 있습니다.
- Parameter에 대한 자세한 설명은 [여기](https://developers.upstage.ai/docs/apis/chat#request-body)를 참고해주세요.

In [76]:
def summarization(dialogue):
    summary = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=build_prompt(dialogue),
        temperature=0.2,
        top_p=0.3,
    )

    return summary.choices[0].message.content

Train Dataset을 이용하여 요약이 잘 되는지 확인해 봅니다.

In [24]:
# Train data 중 처음 3개의 대화를 요약합니다.
def test_on_train_data(num_samples=3):
    for idx, row in train_df[:num_samples].iterrows():
        dialogue = row['dialogue']
        summary = summarization(dialogue)
        print(f"Dialogue:\n{dialogue}\n")
        print(f"Pred Summary: {summary}\n")
        print(f"Gold Summary: {row['summary']}\n")
        print("=="*50)

In [ ]:
if __name__ == "__main__":
    test_on_train_data()

Validation Dataset을 이용하여 요약을 진행하고, 성능을 평가해 봅니다.

In [77]:
# Validation data의 대화를 요약하고, 점수를 측정합니다.
def validate(num_samples=-1):
    val_samples = val_df[:num_samples] if num_samples > 0 else val_df
    
    scores = []
    results_list = []
    for idx, row in tqdm(val_samples.iterrows(), total=len(val_samples)):
        dialogue = row['dialogue']
        gold_summary = row['summary']
        topic = row['topic']
        summary = summarization(dialogue)
        results = compute_metrics(summary, row['summary'])
        combined_rouge = (results['rouge-1'] + results['rouge-2'] + results['rouge-l'])/3 * 100
        avg_score = sum(results.values()) / len(results)
        scores.append(avg_score)
        
        result_dict = {
            # "dialogue": dialogue,
            # "topic": topic,
            "pred_summary": summary,
            "gold_summary": gold_summary,
            'metrics': {
                "rouge-1": float(results['rouge-1']),
                "rouge-2": float(results['rouge-2']),
                "rouge-l": float(results['rouge-l']),
                "combined_rouge": float(combined_rouge)
            }
        }
        results_list.append(result_dict)
        # print(f"Dialogue:\n{dialogue}\n")
        # print(f"Topic: {topic}\n")
        print(f"=={idx}==")
        print(f"Pred Summary: {summary}\n")
        print(f"Gold Summary: {gold_summary}\n")
        print(f"ROUGE-1 Score: {results['rouge-1']}")
        print(f"ROUGE-2 Score: {results['rouge-2']}")
        print(f"ROUGE-L Score: {results['rouge-l']}")
        print(f"Combined ROUGE Score: {combined_rouge}\n")
        print("=="*50)
    combined_rouges = [result['metrics']['combined_rouge'] for result in results_list]

    print(f"Validation Average Score: {sum(combined_rouges) / len(combined_rouges)}")

In [ ]:
if __name__ == "__main__":
    validate() # 100개의 validation sample에 대한 요약을 수행합니다.
    
    # 전체 validation data에 대한 요약을 수행하고 싶은 경우 아래와 같이 실행합니다.
    # validate() 

## 2. Solar Chat API로 요약하기
- Solar Chat API을 이용하여 test dataset에 포함된 dialogue를 요약하고 제출용 파일을 생성합니다.

In [ ]:
def inference():
    test_df = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

    summary = []
    start_time = time.time()
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
        dialogue = row['dialogue']
        summary.append(summarization(dialogue))
        
        # Rate limit 방지를 위해 1분 동안 최대 100개의 요청을 보내도록 합니다.
        if (idx + 1) % 100 == 0:
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            if elapsed_time < 60:
                wait_time = 60 - elapsed_time + 5
                print(f"Elapsed time: {elapsed_time:.2f} sec")
                print(f"Waiting for {wait_time} sec")
                time.sleep(wait_time)
            
            start_time = time.time()
    
    output = pd.DataFrame(
        {
            "fname": test_df['fname'],
            "summary" : summary,
        }
    )
    
    if not os.path.exists(RESULT_PATH):
        os.makedirs(RESULT_PATH)
    output.to_csv(os.path.join(RESULT_PATH, "output_solar.csv"), index=False)

    return output

In [ ]:
if __name__ == "__main__":
    output = inference()

In [ ]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

## 3. Prompt Engineering
- Prompt engineering을 통해 요약 성능 향상을 시도합니다.

In [78]:
# Few-shot prompt를 생성하기 위해, train data의 일부를 사용합니다.
few_shot_samples = train_df.sample(1)

sample_dialogue1 = few_shot_samples.iloc[0]['dialogue']
sample_summary1 = few_shot_samples.iloc[0]['summary']

print(f"Sample Dialogue1:\n{sample_dialogue1}\n")
print(f"Sample Summary1: {sample_summary1}\n")

Sample Dialogue1:
#Person1#: 정부에서 이달 말에 민속 행사를 주최할 예정입니다. 그리고 우리 커뮤니티에서 공연을 하기로 했어요.
#Person2#: 어떤 공연인가요? 사자춤?
#Person1#: 그런 거요, 근데 사자춤은 너무 어렵고 위험할 것 같아요.
#Person2#: 네, 사자와 함께 춤을 추게 되죠. 어떻게 생각하세요? 그럼 전에 했던 레인지 댄스는 어때요?
#Person1#: 좋은 생각이에요. 커뮤니티에 있는 모든 사람들을 모을까요?
#Person2#: 아닐 수도 있죠. 은퇴한 분들 위주로 하는 게 좋을 것 같아요.
#Person1#: 이미 레인지댄스팀이 있는 걸로 알고 있는데, 우리가 필요한 건 리허설만 좀 하면 돼요.
#Person2#: 의상은 어때요?
#Person1#: 커뮤니티에서 모금을 하면 되죠. 한 가정당 10위안 정도면 되겠네요.
#Person2#: 새로운 것을 시도해 보세요. 지원 기관을 찾을 수 있어요.
#Person1#: 좋은 생각이에요. 근처에 우리 후원자가 되고 싶어하는 여행사가 있어요. 소수민족 관광을 홍보할 수 있는 완벽한 기회가 될 거예요.

Sample Summary1: #Person1#과 #Person2#는 민속 행사를 위해 레인지 댄스를 하기로 결정했다. 그들은 은퇴한 사람들에게 집중하고 여행사에게 후원을 요청할 것이다.



In [35]:
# Prompt를 생성하는 함수를 수정합니다.
def build_prompt(dialogue):
    system_prompt = "You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue."

    user_prompt = (
        "Following the instructions below, summarize the given document.\n"
        "Instructions:\n"
        "1. Read the provided sample dialogue and corresponding summary.\n"
        "2. Read the dialogue carefully.\n"
        "3. Following the sample's style of summary, provide a concise summary of the given dialogue.\n\n"
        "Sample Dialogue:\n"
        f"{sample_dialogue1}\n\n"
        "Sample Summary:\n"
        f"{sample_summary1}\n\n"
        "Dialogue:\n"
        f"{dialogue}\n\n"
        "Summary:\n"
    )
    
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]

In [ ]:
# 변경된 prompt를 사용하여, train data 중 처음 3개의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    test_on_train_data()

In [ ]:
# 변경된 prompt를 사용하여, validation data의 대화를 요약하고, 점수를 측정합니다.
if __name__ == "__main__":
    validate()

다른 방식으로 Few-shot sample을 제공하여 Prompt를 구성해 봅니다.

In [79]:
# Few-shot sample을 다른 방식으로 사용하여 prompt를 생성합니다.
def build_prompt(dialogue):
    system_prompt = "You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue."

    few_shot_user_prompt_1 = (
        "Following the instructions below, summarize the given document.\n"
        "Instructions:\n"
        "1. Read the provided sample dialogue and corresponding summary.\n"
        "2. Read the dialogue carefully.\n"
        "3. Following the sample's style of summary, provide a concise summary of the given dialogue. Be sure that the summary is simple but captures the essence of the dialogue.\n\n"
        "Dialogue:\n"
        f"{sample_dialogue1}\n\n"
        "Summary:\n"
    )
    few_shot_assistant_prompt_1 = sample_summary1
    
    user_prompt = (
        "Dialogue:\n"
        f"{dialogue}\n\n"
        "Summary:\n"
    )
    
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": few_shot_user_prompt_1},
        {"role": "assistant", "content": few_shot_assistant_prompt_1},
        {"role": "user", "content": user_prompt},
    ]

In [ ]:
# 변경된 prompt를 사용하여, train data 중 처음 3개의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    test_on_train_data()

In [80]:
# 변경된 prompt를 사용하여, validation data의 대화를 요약하고, 점수를 측정합니다.
if __name__ == "__main__":
    validate()

  0%|          | 1/499 [00:01<10:38,  1.28s/it]

==0==
Pred Summary: #Person2#는 숨쉬기에 어려움을 겪고 있으며, 특히 운동할 때 증상이 심해진다. #Person1#은 #Person2#를 폐 전문의에게 보내 천식 검사를 받도록 할 것이다.

Gold Summary: #Person2#는 숨쉬기에 어려움을 겪는다. 의사는 #Person1#에게 이에 대해 묻고, #Person2#를 폐 전문의에게 보낼 예정이다. 

ROUGE-1 Score: 0.3428571379591837
ROUGE-2 Score: 0.24242423753902673
ROUGE-L Score: 0.3428571379591837
Combined ROUGE Score: 30.937950448579805



  0%|          | 2/499 [00:01<07:34,  1.09it/s]

==1==
Pred Summary: #Person1#과 #Person2#는 운동 계획을 변경하여 오늘은 팔과 배를 운동하고 금요일에 다리를 운동하기로 합의했다.

Gold Summary: #Person1#은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.

ROUGE-1 Score: 0.18181817698347122
ROUGE-2 Score: 0.09999999520000023
ROUGE-L Score: 0.18181817698347122
Combined ROUGE Score: 15.454544972231421



  1%|          | 3/499 [00:02<07:24,  1.12it/s]

...

==496==
Pred Summary: #Person1#은 #Person2#의 차에서 이상한 소리가 나는 것을 브레이크 교체가 필요한 것으로 진단했다. #Person2#는 오늘 차를 돌려받을 수 없다는 사실에 실망했지만, #Person1#은 안전을 위해 차를 운전하지 말고 버스를 타는 것을 제안했다.

Gold Summary: #Person2#의 차에서 이상한 소리가 납니다. #Person1#는 브레이크를 교체해야 할 것으로 생각하지만, #Person1#는 내일까지 그것을 고칠 수 없습니다. #Person2#는 오늘 밤에 공연을 보러 가고 싶어합니다; #Person1#는 #Person2#에게 버스를 이용할 것을 제안합니다.

ROUGE-1 Score: 0.3636363586380166
ROUGE-2 Score: 0.1379310294827588
ROUGE-L Score: 0.3636363586380166
Combined ROUGE Score: 28.840124891959736



100%|█████████▉| 498/499 [07:28<00:00,  1.15it/s]

==497==
Pred Summary: 고객은 아마존에서 구매한 책에서 53페이지가 빠져있음을 알렸다. 아마존은 고객이 빠진 부분을 사진으로 찍어 업로드하면 2일 내로 새 책을 보내주겠다고 안내했다. 고객은 이전 책을 가지고 있어도 된다.

Gold Summary: #Person2#님이 아마존 고객 서비스에 전화하여 아마존에서 받은 책에 한 페이지가 빠져 있다고 합니다. #Person1#은 문제가 확인되면 새 책을 보내드릴 것이라고 말합니다.

ROUGE-1 Score: 0.13333332835555575
ROUGE-2 Score: 0.04444443950617339
ROUGE-L Score: 0.13333332835555575
Combined ROUGE Score: 10.370369873909496



100%|██████████| 499/499 [07:29<00:00,  1.11it/s]

==498==
Pred Summary: #Person1#과 #Person2#는 여름이 빠르게 지나가는 것에 대해 이야기합니다. #Person2#는 여름 휴가 동안 파티를 도와주는 회사에서 일할 것이라고 말합니다. 그들은 음식 준비, 서빙, 청소 등을 도와줍니다. #Person1#은 #Person2#의 직업이 꽤 멋져 보인다고 생각합니다.

Gold Summary: #Person2#는 #Person1#에게 여름 휴가 동안 파티를 도와주는 회사에서 일할 것이라고 말한다. #Person1#는 그것이 멋진 직업이라고 생각한다.

ROUGE-1 Score: 0.38297871891353563
ROUGE-2 Score: 0.30434782169187147
ROUGE-L Score: 0.38297871891353563
Combined ROUGE Score: 35.67684198396475

Validation Average Score: 17.67771527868936


### (선택) 변경된 Prompt로 test dataset에 대한 요약을 진행합니다.
- 변경된 prompt를 통해 점수가 개선되었다면, test dataset에 대한 요약을 진행하고 제출합니다.

In [ ]:
# 변경된 prompt를 사용하여, test data의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    output = inference()

In [ ]:
output